In [1]:
import glob
import pandas as pd
import numpy as np
import imageio
from skimage.transform import resize
from scipy import misc

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.utils import normalize
from keras.backend import set_floatx
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, TensorBoard
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
import pickle

In [5]:
PATH_DATA = './Data/'
PATH_IMAGES = glob.glob('./Images/*.jpg')

In [6]:
df_meta = pd.read_csv(PATH_DATA + 'HAM10000_metadata.csv')

## I. EDA

In [7]:
df_meta.shape

(10015, 7)

In [8]:
df_meta.sample()

,lesion_id,image_id,dx,dx_type,age,sex,localization
307,HAM_0001995,ISIC_0024581,bkl,histo,40.0,male,lower extremity


In [9]:
df_meta['dx'].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [10]:
df_meta['dx_type'].value_counts()

histo        5340
follow_up    3704
consensus     902
confocal       69
Name: dx_type, dtype: int64

In [11]:
df_meta['age'].value_counts()

45.0    1299
50.0    1187
55.0    1009
40.0     985
60.0     803
70.0     756
35.0     753
65.0     731
75.0     618
30.0     464
80.0     404
85.0     290
25.0     247
20.0     169
5.0       86
15.0      77
10.0      41
0.0       39
Name: age, dtype: int64

In [12]:
df_meta['sex'].value_counts()

male       5406
female     4552
unknown      57
Name: sex, dtype: int64

In [13]:
df_meta['localization'].value_counts()

back               2192
lower extremity    2077
trunk              1404
upper extremity    1118
abdomen            1022
face                745
chest               407
foot                319
unknown             234
neck                168
scalp               128
hand                 90
ear                  56
genital              48
acral                 7
Name: localization, dtype: int64

## Experimentation
### Method 1 results in dead kernel

In [14]:
# Method 1: Load all Images and Scale by 255
# images = []
# for path_image in PATH_IMAGES:
#     image = misc.imread(path_image)
#     image = image / 255
#     images.append(image)

### Method 2 can be completed with approx 3GB memory consumption

In [15]:
# Method 2: Load all Images and Resize to 50%, no scaling
# images = []
# for path_image in PATH_IMAGES:
#     image = misc.imread(path_image)
#     image = misc.imresize(image, size=(300, 225), interp='nearest')
#     images.append(image)
#     print('Completed processing {}'.format(path_image))

### Method 3 can be completed with approx 5GB memory consumption

In [16]:
# Method 3: Load all Images and Resize to 70%, no scaling
# images = []
# for path_image in PATH_IMAGES:
#     image = misc.imread(path_image)
#     image = misc.imresize(image, size=(420, 315), interp='nearest')
#     images.append(image)
#     print('Completed processing {}'.format(path_image))

In [17]:
# images = np.asarray(images)

In [18]:
# np.asarray([images.shape[1], images.shape[2], images.shape[3]])
# array([420, 315,   3])

### Method 4 scaling values takes up entire 16GB + swap

In [19]:
# Method 4: Load all Images and Resize to 50%, with scaling
# images = []
# for path_image in PATH_IMAGES:
#     image = misc.imread(path_image)
#     image = misc.imresize(image, size=(300, 225), interp='nearest')
#     image = image / 255
#     images.append(image)

## Data Visualisation

In [20]:
# This is working, redo this later
# figure = plt.figure()
# count = 0
# for index, row in df_combined.iterrows():
#     if count < 10:
#         plt.imshow(row['image'])
#         plt.axis('off')
#         count += 1
#     else:
#         break

## Modeling - CNN1

In [21]:
# Prepare Train and Test Data
df_meta = df_meta.set_index('image_id')
# df_meta = df_meta.sample(100)
y = df_meta['dx']
y_train, y_test = train_test_split(y, test_size=0.20, random_state=42)

In [40]:
y_train2, y_test2 = train_test_split(y, test_size=0.20, random_state=42)
pd.Series(y_test2).value_counts()

nv       1338
bkl       228
mel       226
bcc        93
akiec      69
df         28
vasc       21
Name: dx, dtype: int64

In [41]:
pd.Series(y_train2).value_counts()

nv       5367
mel       887
bkl       871
bcc       421
akiec     258
vasc      121
df         87
Name: dx, dtype: int64

In [22]:
# Original Size: 600*450
ORIG_SIZE = (600, 450)
MODD_SIZE = (300, 225)

# Method 3: Load all Images and Resize to 50% and normalized - can complete 6GB RAM
x_train, x_test = [], []
for image_id in y_train.index.values:
    #Uses too much memory, will revert first to scipy
    #image = imageio.imread('./Images/{}.jpg'.format(image_id))
    #image = resize(image, output_shape=(360, 270), anti_aliasing=False, mode='constant')
    image = misc.imread('./Images/{}.jpg'.format(image_id))
    image = misc.imresize(image, size=MODD_SIZE, interp='nearest')
    image = normalize(np.float16(image))
    x_train.append(image)
    print('Completed processing {}.jpg'.format(image_id))

x_train = np.asarray(x_train)

for image_id in y_test.index.values:
    #Uses too much memory, will revert first to scipy
    #image = imageio.imread('./Images/{}.jpg'.format(image_id))
    #image = resize(image, output_shape=(360, 270), anti_aliasing=False, mode='constant')
    image = misc.imread('./Images/{}.jpg'.format(image_id))
    image = misc.imresize(image, size=MODD_SIZE, interp='nearest')
    image = normalize(np.float16(image))
    x_test.append(image)
    print('Completed processing {}.jpg'.format(image_id))

x_test = np.asarray(x_test)

/home/neil/anaconda3/envs/Creadits/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # This is added back by InteractiveShellApp.init_path()
/home/neil/anaconda3/envs/Creadits/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':
/home/neil/anaconda3/envs/Creadits/lib/python3.6/site-packages/numpy/linalg/linalg.py:2390: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


Completed processing ISIC_0027223.jpg
Completed processing ISIC_0032914.jpg
Completed processing ISIC_0033614.jpg
Completed processing ISIC_0026219.jpg
Completed processing ISIC_0033900.jpg
Completed processing ISIC_0031846.jpg
Completed processing ISIC_0032163.jpg
Completed processing ISIC_0033945.jpg
Completed processing ISIC_0032329.jpg
Completed processing ISIC_0025860.jpg
Completed processing ISIC_0027338.jpg
Completed processing ISIC_0032605.jpg
Completed processing ISIC_0027008.jpg
Completed processing ISIC_0030154.jpg
Completed processing ISIC_0024585.jpg
Completed processing ISIC_0034064.jpg
Completed processing ISIC_0033560.jpg
Completed processing ISIC_0024859.jpg
Completed processing ISIC_0031458.jpg
Completed processing ISIC_0029097.jpg
Completed processing ISIC_0025782.jpg
Completed processing ISIC_0030176.jpg
Completed processing ISIC_0031785.jpg
Completed processing ISIC_0026954.jpg
Completed processing ISIC_0032986.jpg
Completed processing ISIC_0033601.jpg
Completed pr

Completed processing ISIC_0027648.jpg
Completed processing ISIC_0024597.jpg
Completed processing ISIC_0027013.jpg
Completed processing ISIC_0033233.jpg
Completed processing ISIC_0026755.jpg
Completed processing ISIC_0029547.jpg
Completed processing ISIC_0032844.jpg
Completed processing ISIC_0025825.jpg
Completed processing ISIC_0034271.jpg
Completed processing ISIC_0024995.jpg
Completed processing ISIC_0025948.jpg
Completed processing ISIC_0029722.jpg
Completed processing ISIC_0032565.jpg
Completed processing ISIC_0025898.jpg
Completed processing ISIC_0028466.jpg
Completed processing ISIC_0026361.jpg
Completed processing ISIC_0034222.jpg
Completed processing ISIC_0031884.jpg
Completed processing ISIC_0034149.jpg
Completed processing ISIC_0028416.jpg
Completed processing ISIC_0025513.jpg
Completed processing ISIC_0031995.jpg
Completed processing ISIC_0026764.jpg
Completed processing ISIC_0030521.jpg
Completed processing ISIC_0025642.jpg
Completed processing ISIC_0034203.jpg
Completed pr

Completed processing ISIC_0028129.jpg
Completed processing ISIC_0025118.jpg
Completed processing ISIC_0031030.jpg
Completed processing ISIC_0030757.jpg
Completed processing ISIC_0025341.jpg
Completed processing ISIC_0032204.jpg
Completed processing ISIC_0027101.jpg
Completed processing ISIC_0026832.jpg
Completed processing ISIC_0029763.jpg
Completed processing ISIC_0029481.jpg
Completed processing ISIC_0029466.jpg
Completed processing ISIC_0033180.jpg
Completed processing ISIC_0033677.jpg
Completed processing ISIC_0032739.jpg
Completed processing ISIC_0026916.jpg
Completed processing ISIC_0029988.jpg
Completed processing ISIC_0033944.jpg
Completed processing ISIC_0028678.jpg
Completed processing ISIC_0032465.jpg
Completed processing ISIC_0028262.jpg
Completed processing ISIC_0032231.jpg
Completed processing ISIC_0033399.jpg
Completed processing ISIC_0030939.jpg
Completed processing ISIC_0031332.jpg
Completed processing ISIC_0026166.jpg
Completed processing ISIC_0025065.jpg
Completed pr

Completed processing ISIC_0031593.jpg
Completed processing ISIC_0031015.jpg
Completed processing ISIC_0026782.jpg
Completed processing ISIC_0030356.jpg
Completed processing ISIC_0030274.jpg
Completed processing ISIC_0024855.jpg
Completed processing ISIC_0029812.jpg
Completed processing ISIC_0024945.jpg
Completed processing ISIC_0031626.jpg
Completed processing ISIC_0028777.jpg
Completed processing ISIC_0024337.jpg
Completed processing ISIC_0024538.jpg
Completed processing ISIC_0026134.jpg
Completed processing ISIC_0027488.jpg
Completed processing ISIC_0030758.jpg
Completed processing ISIC_0029115.jpg
Completed processing ISIC_0025725.jpg
Completed processing ISIC_0031606.jpg
Completed processing ISIC_0026455.jpg
Completed processing ISIC_0027112.jpg
Completed processing ISIC_0026168.jpg
Completed processing ISIC_0029352.jpg
Completed processing ISIC_0031555.jpg
Completed processing ISIC_0028938.jpg
Completed processing ISIC_0025587.jpg
Completed processing ISIC_0025890.jpg
Completed pr

...

Completed processing ISIC_0028118.jpg
Completed processing ISIC_0025036.jpg
Completed processing ISIC_0031401.jpg
Completed processing ISIC_0032817.jpg
Completed processing ISIC_0027152.jpg
Completed processing ISIC_0031811.jpg
Completed processing ISIC_0028725.jpg
Completed processing ISIC_0027467.jpg
Completed processing ISIC_0026239.jpg
Completed processing ISIC_0029911.jpg
Completed processing ISIC_0028276.jpg
Completed processing ISIC_0028023.jpg
Completed processing ISIC_0033350.jpg
Completed processing ISIC_0026359.jpg
Completed processing ISIC_0030536.jpg
Completed processing ISIC_0029411.jpg
Completed processing ISIC_0027427.jpg
Completed processing ISIC_0032578.jpg
Completed processing ISIC_0029900.jpg
Completed processing ISIC_0030987.jpg
Completed processing ISIC_0026376.jpg
Completed processing ISIC_0029331.jpg
Completed processing ISIC_0027028.jpg
Completed processing ISIC_0032003.jpg
Completed processing ISIC_0026952.jpg
Completed processing ISIC_0031810.jpg
Completed pr

Completed processing ISIC_0029552.jpg
Completed processing ISIC_0024460.jpg
Completed processing ISIC_0024528.jpg
Completed processing ISIC_0033171.jpg
Completed processing ISIC_0027065.jpg
Completed processing ISIC_0027392.jpg
Completed processing ISIC_0030728.jpg
Completed processing ISIC_0030354.jpg
Completed processing ISIC_0028688.jpg
Completed processing ISIC_0032396.jpg
Completed processing ISIC_0031706.jpg
Completed processing ISIC_0030220.jpg
Completed processing ISIC_0026380.jpg
Completed processing ISIC_0026536.jpg
Completed processing ISIC_0030739.jpg
Completed processing ISIC_0030263.jpg
Completed processing ISIC_0027623.jpg
Completed processing ISIC_0025345.jpg
Completed processing ISIC_0026538.jpg
Completed processing ISIC_0032096.jpg
Completed processing ISIC_0033600.jpg
Completed processing ISIC_0026878.jpg
Completed processing ISIC_0027222.jpg
Completed processing ISIC_0028639.jpg
Completed processing ISIC_0025128.jpg
Completed processing ISIC_0027362.jpg
Completed pr

Completed processing ISIC_0028052.jpg
Completed processing ISIC_0029638.jpg
Completed processing ISIC_0032608.jpg
Completed processing ISIC_0028281.jpg
Completed processing ISIC_0031605.jpg
Completed processing ISIC_0028763.jpg
Completed processing ISIC_0026527.jpg
Completed processing ISIC_0026342.jpg
Completed processing ISIC_0033616.jpg
Completed processing ISIC_0030992.jpg
Completed processing ISIC_0028979.jpg
Completed processing ISIC_0033656.jpg
Completed processing ISIC_0024851.jpg
Completed processing ISIC_0028524.jpg
Completed processing ISIC_0032942.jpg
Completed processing ISIC_0033875.jpg
Completed processing ISIC_0032793.jpg
Completed processing ISIC_0030813.jpg
Completed processing ISIC_0028966.jpg
Completed processing ISIC_0032551.jpg
Completed processing ISIC_0032430.jpg
Completed processing ISIC_0031582.jpg
Completed processing ISIC_0033965.jpg
Completed processing ISIC_0028824.jpg
Completed processing ISIC_0026504.jpg
Completed processing ISIC_0029427.jpg
Completed pr

Completed processing ISIC_0030061.jpg
Completed processing ISIC_0027247.jpg
Completed processing ISIC_0024821.jpg
Completed processing ISIC_0029200.jpg
Completed processing ISIC_0030661.jpg
Completed processing ISIC_0032368.jpg
Completed processing ISIC_0027736.jpg
Completed processing ISIC_0033665.jpg
Completed processing ISIC_0032854.jpg
Completed processing ISIC_0028207.jpg
Completed processing ISIC_0033246.jpg
Completed processing ISIC_0024906.jpg
Completed processing ISIC_0026095.jpg
Completed processing ISIC_0031993.jpg
Completed processing ISIC_0032497.jpg
Completed processing ISIC_0034053.jpg
Completed processing ISIC_0025962.jpg
Completed processing ISIC_0031549.jpg
Completed processing ISIC_0029612.jpg
Completed processing ISIC_0025086.jpg
Completed processing ISIC_0030302.jpg
Completed processing ISIC_0027697.jpg
Completed processing ISIC_0024400.jpg
Completed processing ISIC_0028853.jpg
Completed processing ISIC_0029935.jpg
Completed processing ISIC_0030166.jpg
Completed pr

Completed processing ISIC_0027749.jpg
Completed processing ISIC_0031455.jpg
Completed processing ISIC_0031029.jpg
Completed processing ISIC_0030238.jpg
Completed processing ISIC_0034317.jpg
Completed processing ISIC_0032245.jpg
Completed processing ISIC_0033779.jpg
Completed processing ISIC_0029949.jpg


In [23]:
y_train = y_train.values
y_test = y_test.values

In [24]:
def one_hot_encode_object_array(arr):
    # One hot encode a numpy array of objects (e.g. strings)'''
    uniques, ids = np.unique(arr, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))

y_train = one_hot_encode_object_array(y_train)
y_test = one_hot_encode_object_array(y_test)

In [25]:
# set_floatx('float16')

In [26]:
def cnn(size, n_layers):
    # INPUTS
    # size     - size of the input images
    # n_layers - number of layers
    # OUTPUTS
    # model    - compiled CNN

    # Define hyperparamters
    MIN_NEURONS = 20
    MAX_NEURONS = 120
    KERNEL = (3, 3)

    # Determine the # of neurons in each convolutional layer
    steps = np.floor(MAX_NEURONS / (n_layers + 1))
    nuerons = np.arange(MIN_NEURONS, MAX_NEURONS, steps)
    nuerons = nuerons.astype(np.int32)

    # Define a model
    model = Sequential()

    # Add convolutional layers
    for i in range(0, n_layers):
        if i == 0:
            shape = (size[0], size[1], size[2])
            model.add(Conv2D(nuerons[i], KERNEL, input_shape=shape))
        else:
            model.add(Conv2D(nuerons[i], KERNEL))

        model.add(Activation('relu'))
        model.add(Dropout(0.25))

    # Add max pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(MAX_NEURONS))
    model.add(Activation('relu'))

    # Add output layer
    model.add(Dense(7)) # -- change this
    model.add(Activation('softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Print a summary of the model
    model.summary()

    return model

In [27]:
# Instantiate the model
N_LAYERS = 2
image_size = np.asarray([x_train.shape[1], x_train.shape[2], x_train.shape[3]])
model = cnn(size=image_size, n_layers=N_LAYERS)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 223, 20)      560       
_________________________________________________________________
activation_1 (Activation)    (None, 298, 223, 20)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 298, 223, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 296, 221, 60)      10860     
_________________________________________________________________
activation_2 (Activation)    (None, 296, 221, 60)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 296, 221, 60)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 110, 60)      0         
__________

In [28]:
# Training hyperparamters
EPOCHS = 16
BATCH_SIZE = 16

In [29]:
# Train the model
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)

In [30]:
# Make a prediction on the test set
test_predictions = model.predict(x_test)
test_predictions = np.round(test_predictions)

In [31]:
# Report the accuracy
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy: " + str(accuracy))

Accuracy: 0.6679980029955067


In [32]:
model.evaluate(x_train, y_train, BATCH_SIZE, verbose=0)

[5.3210637111159125, 0.6698701947376958]

In [33]:
# Save the Model
# model_cnn1_file = open('model-cnn1.sav','wb')
# pickle.dump(model, model_cnn1_file)
# model_cnn1_file.close()

In [35]:
print(classification_report(y_test, test_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        69
           1       0.00      0.00      0.00        93
           2       0.00      0.00      0.00       228
           3       0.00      0.00      0.00        28
           4       0.00      0.00      0.00       226
           5       0.67      1.00      0.80      1338
           6       0.00      0.00      0.00        21

   micro avg       0.67      0.67      0.67      2003
   macro avg       0.10      0.14      0.11      2003
weighted avg       0.45      0.67      0.54      2003
 samples avg       0.67      0.67      0.67      2003



/home/neil/anaconda3/envs/Creadits/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/neil/anaconda3/envs/Creadits/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/neil/anaconda3/envs/Creadits/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# Overall Findings
- Dataset is hugely imbalanced
- Need to create 2 models and address the minority